In [ ]:
%pip install torchvision
%pip install pytorch-lightning

In [2]:
from joblib import Parallel, delayed
import joblib
import nltk
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import defaultdict,Counter
import re
from nltk.stem import WordNetLemmatizer,PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


In [3]:
nltk.download('stopwords')
stop=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Computing\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import time
import csv
import os
import json


# Preinstalled packages
import requests
import pandas as pd


In [5]:
news_2021_2022  = pd.read_csv("../archive/cryptonews_sentiment.csv")

In [8]:
news_2021_2022 = news_2021_2022[['date','sentiment','title']]

In [ ]:
news_2021_2022

In [10]:
news_2013_2018  = pd.read_csv("../archive/new_news_processed_sentiment.csv")

In [14]:
news_2013_2018 = news_2013_2018[['date','sentiment','title']]

In [ ]:
news_2013_2018

In [16]:
frames = [news_2013_2018,news_2021_2022]
news = pd.concat(frames,ignore_index=True)

In [ ]:
news 

In [17]:
news.to_csv('../archive/cryptonews_processed_2013_2022.csv',index=False)

In [5]:
news_2013_2017 = pd.read_csv("../archive/crypto_news_parsed_2013-2017_train.csv")
news_2018 = pd.read_csv("../archive/crypto_news_parsed_2018_validation.csv")
frames = [news_2013_2017,news_2018 ]
news = pd.concat(frames,ignore_index=True)

In [6]:
news.shape

(39308, 7)

In [8]:
news.head()

,url,title,text,html,year,author,source
0,https://www.ccn.com/bitcoin-price-update-will-...,Bitcoin Price Update: Will China Lead us Down?,Bitcoin Priced in USD on Mt. GoxAbove is Gox p...,"<figure id=""attachment_4090"" style=""width: 838...",2013,Gordon Hall,altcoin_analysis
1,https://www.ccn.com/key-bitcoin-price-levels-w...,Key Bitcoin Price Levels for Week 51 (15 – 22 ...,"The Bitcoin price up to 2013/12/14, with Fib l...","<figure id=""attachment_3961"" style=""width: 838...",2013,Gordon Hall,altcoin_analysis
2,https://www.ccn.com/national-australia-bank-ci...,"National Australia Bank, Citing Highly Flawed ...",National Australia Bank looms over Melbourne’s...,"<figure id=""attachment_5011"" style=""width: 750...",2013,Gordon Hall,altcoin_analysis
3,https://www.ccn.com/chinese-banking-ban-bitcoi...,Chinese Bitcoin Ban Driven by Chinese Banking...,Recent reports from China indicate there may b...,"<p><span style=""line-height: 1.5em;"">Recent re...",2013,Jonas Borchgrevink,altcoin_analysis
4,https://www.ccn.com/bitcoin-trade-update-opene...,Bitcoin Trade Update: Opened Position,"Opened 1st Bitcoin trade position, price curre...","<figure id=""attachment_4921"" style=""width: 750...",2013,Gordon Hall,altcoin_analysis


In [7]:
def find_date_from_html(year, html):
    date_regex = str(year) + r'[-/](\d{2})[-/](\d{2})'
    date_matches = re.findall(date_regex,html)
    date_matches = [(int(m),int(d)) for m,d in date_matches if int(m) < 13 and int(d) <= monthrange(int(year), int(m))[1] and int(d) > 0 and int(m) > 0]
    if not date_matches:
        return (None, None)
    return max(set(date_matches), key=date_matches.count)

In [9]:
news['url'].dtype

dtype('O')

In [10]:
len(news)

39308

In [13]:
news[news['year']==2012]

,url,title,text,html,year,author,source


In [18]:
from calendar import monthrange
import numpy as np
import pandas as pd
import seaborn as sns
import re
import pickle
from datetime import datetime, timedelta, date
import pytz
import torch
import torch.nn as nn
import torch.utils.data as td
import sklearn
import pytorch_lightning as pl
from typing import List, Optional

date=[]
count=0
for i in range(0,len(news)):
    date.append(find_date_from_html(news['year'][i],news['url'][i]))
    if date[i] == (None,None):
        count+=1

In [19]:
count

29795

In [20]:
news[["month", "day"]] = news.apply(lambda x: find_date_from_html(x["year"], x["html"]), axis=1, result_type='expand')
news['date'] = pd.to_datetime(news[["year", "month", "day"]], utc=True)
news = news.drop(columns=["year", "month", "day"])
#news = news.set_index("date")
news.dropna()

,url,title,text,html,author,source,date
0,https://www.ccn.com/bitcoin-price-update-will-...,Bitcoin Price Update: Will China Lead us Down?,Bitcoin Priced in USD on Mt. GoxAbove is Gox p...,"<figure id=""attachment_4090"" style=""width: 838...",Gordon Hall,altcoin_analysis,2013-12-16 00:00:00+00:00
1,https://www.ccn.com/key-bitcoin-price-levels-w...,Key Bitcoin Price Levels for Week 51 (15 – 22 ...,"The Bitcoin price up to 2013/12/14, with Fib l...","<figure id=""attachment_3961"" style=""width: 838...",Gordon Hall,altcoin_analysis,2013-12-14 00:00:00+00:00
2,https://www.ccn.com/national-australia-bank-ci...,"National Australia Bank, Citing Highly Flawed ...",National Australia Bank looms over Melbourne’s...,"<figure id=""attachment_5011"" style=""width: 750...",Gordon Hall,altcoin_analysis,2013-12-20 00:00:00+00:00
4,https://www.ccn.com/bitcoin-trade-update-opene...,Bitcoin Trade Update: Opened Position,"Opened 1st Bitcoin trade position, price curre...","<figure id=""attachment_4921"" style=""width: 750...",Gordon Hall,altcoin_analysis,2013-12-21 00:00:00+00:00
5,https://www.ccn.com/key-bitcoin-price-levels-w...,Key Bitcoin Price Levels for Week 52 (22 – 28 ...,"Gox Bitcoin Price, Volume, Fib Levels, trendli...","<figure id=""attachment_4520"" style=""width: 192...",Gordon Hall,altcoin_analysis,2013-12-05 00:00:00+00:00
...,...,...,...,...,...,...,...
39262,https://cointelegraph.com/news/indie-game-deli...,Indie Game Delisted From Steam Marketplace for...,Valve Corporation has removed a game from its ...,"<p dir=""ltr"">Valve Corporation has removed a g...",Ana Alexandre,scam_news,2018-07-30 00:00:00+00:00
39279,https://cointelegraph.com/news/korean-crypto-e...,Korean Crypto Exchange Execs Detained On Fraud...,CoinNest co-founder and chief executive Kim Ik...,"<p dir=""ltr""><a href=""https://www.coinnest.co....",Marie Huillet,scam_news,2018-01-23 00:00:00+00:00
39291,https://cointelegraph.com/news/steve-wozniak-l...,Steve Wozniak ‘Loses 7 BTC’ In Unlikely Credit...,Apple co-founder Steve Wozniak allegedly lost ...,"<p dir=""ltr"">Apple co-founder Steve Wozniak al...",William Suberg,scam_news,2018-02-26 00:00:00+00:00
39298,https://cointelegraph.com/news/opera-browser-a...,"Opera Browser Addresses Cryptojacking, Adds An...","Opera, a web browser that has since December i...","<p dir=""ltr"">Opera, a web browser that has sin...",Molly Jane Zuckerman,scam_news,2018-01-22 00:00:00+00:00


In [21]:
news['date'].unique()

<DatetimeArray>
['2013-12-16 00:00:00+00:00', '2013-12-14 00:00:00+00:00',
 '2013-12-20 00:00:00+00:00',                       'NaT',
 '2013-12-21 00:00:00+00:00', '2013-12-05 00:00:00+00:00',
 '2013-12-19 00:00:00+00:00', '2013-12-07 00:00:00+00:00',
 '2013-12-27 00:00:00+00:00', '2013-12-13 00:00:00+00:00',
 ...
 '2018-08-10 00:00:00+00:00', '2018-11-13 00:00:00+00:00',
 '2018-01-13 00:00:00+00:00', '2018-03-17 00:00:00+00:00',
 '2018-03-03 00:00:00+00:00', '2018-07-14 00:00:00+00:00',
 '2018-04-07 00:00:00+00:00', '2018-01-20 00:00:00+00:00',
 '2018-04-28 00:00:00+00:00', '2018-07-21 00:00:00+00:00']
Length: 1736, dtype: datetime64[ns, UTC]

In [22]:
new_news=news[news['date'].isna()!=True]

In [23]:
new_news['date'].value_counts()

date
2017-12-31 00:00:00+00:00    110
2018-04-04 00:00:00+00:00     59
2018-05-15 00:00:00+00:00     56
2018-02-20 00:00:00+00:00     50
2018-03-29 00:00:00+00:00     49
                            ... 
2013-12-02 00:00:00+00:00      1
2013-09-18 00:00:00+00:00      1
2013-06-18 00:00:00+00:00      1
2013-09-25 00:00:00+00:00      1
2014-05-11 00:00:00+00:00      1
Name: count, Length: 1735, dtype: int64

In [24]:
new_news=new_news.sort_values(by='date')

In [26]:
new_news.shape


(15350, 7)

In [28]:
new_news


,url,title,text,html,author,source,date
508,https://www.coindesk.com/bitcoin-exchange-btc-...,Bitcoin exchange BTC China temporarily halts t...,BTC China has become the world's first major b...,<p>BTC China has become the world's first majo...,Emily Spaven,news,2013-01-01 00:00:00+00:00
271,https://www.coindesk.com/what-happened-at-the-...,What happened at the US regulators' bitcoin me...,The Bitcoin Foundation's meeting with represen...,"<p>The Bitcoin Foundation's <a href=""https://w...",Emily Spaven,news,2013-01-01 00:00:00+00:00
593,https://www.coindesk.com/vault-satoshi-expands...,Vault of Satoshi expands Canadian bitcoin exch...,"A new Canadian exchange launched this week, of...","<p>A new Canadian exchange launched this week,...",Danny Bradbury,news,2013-01-02 00:00:00+00:00
175,https://www.coindesk.com/what-will-gambling-do...,What will gambling do to the bitcoin block chain?,What are gambling transactions doing to the bl...,<p>What are gambling transactions doing to the...,Danny Bradbury,news,2013-01-22 00:00:00+00:00
616,https://www.coindesk.com/bitmarkers-move-money...,Bitmarkers wants to move money in bitcoin for ...,"At first blush, bitcoin and gambling seem like...","<p>At first blush, bitcoin and gambling seem l...",Daniel Cawrey,news,2013-01-26 00:00:00+00:00
...,...,...,...,...,...,...,...
39158,https://cointelegraph.com/news/fifth-largest-c...,Fifth Largest Crypto Exchange Bithumb to Reope...,"Bithumb, one of the largest South Korean crypt...","<p dir=""ltr""><a data-amp=""https://cointelegrap...",William Suberg,regulation_news,2018-08-29 00:00:00+00:00
31652,https://www.ccn.com/bitcoin-price-intraday-ana...,Bitcoin Price Intraday Analysis: BTC/USD Locke...,Bitcoin price today remained in a bias-conflic...,<p>Bitcoin price today remained in a bias-conf...,Yashu Gola,bitcoin_analysis,2018-08-29 00:00:00+00:00
29949,https://www.newsbtc.com/2018/08/29/ripple-xrp-...,Ripple (XRP) Price Watch: Rejected at Range Re...,Ripple Price Key HighlightsRipple has been on ...,"<span class=""entry-content""><p><strong>Ripple...",Sarah Jenn,analysis,2018-08-29 00:00:00+00:00
28111,https://www.ccn.com/architect-launches-bitcoin...,Architect Launches Bitcoin Hostel Near Rio De ...,"Alessandro Santos, a Brazilian architect, engi...","<p>Alessandro Santos, a Brazilian architect, e...",Francisco Memoria,accepts_bitcoin,2018-10-01 00:00:00+00:00


In [29]:
new_news.to_csv('../archive/new_news.csv',index=False)

In [30]:
#Remove Urls and HTML links
def remove_urls(text):
    url_remove = re.compile(r'https?://\S+|www\.\S+')
    return url_remove.sub(r'', text)
new_news['title']=new_news['title'].apply(lambda x:remove_urls(x))

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)
new_news['title']=new_news['title'].apply(lambda x:remove_html(x))

In [31]:
# Lower casing
def lower(text):
    low_text= text.lower()
    return low_text
new_news['title']=new_news['title'].apply(lambda x:lower(x))


# Number removal
def remove_num(text):
    remove= re.sub(r'\d+', '', text)
    return remove
new_news['title']=new_news['title'].apply(lambda x:remove_num(x))

In [32]:
#Remove stopwords & Punctuations
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))

def punct_remove(text):
    punct = re.sub(r"[^\w\s\d]","", text)
    return punct
new_news['title']=new_news['title'].apply(lambda x:punct_remove(x))



def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
new_news['title']=new_news['title'].apply(lambda x:remove_stopwords(x))

In [33]:
#Remove mentions and hashtags
def remove_mention(x):
    text=re.sub(r'@\w+','',x)
    return text
new_news['title']=new_news['title'].apply(lambda x:remove_mention(x))
def remove_hash(x):
    text=re.sub(r'#\w+','',x)
    return text
new_news['title']=new_news['title'].apply(lambda x:remove_hash(x))

In [34]:
#Remove extra white space left while removing stuff
def remove_space(text):
    space_remove = re.sub(r"\s+"," ",text).strip()
    return space_remove
def stemming(text):
    ps = PorterStemmer()
    
    return" ".join([ps.stem(word) for word in str(text).split()])
    
def lemmatizing(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    
    return" ".join([wordnet_lemmatizer.lemmatize(word) for word in str(text).split()])
new_news['title']=new_news['title'].apply(lambda x:remove_space(x))

In [35]:
from collections import Counter
comment_words = '' 
stopwords = set(STOPWORDS) 

for val in new_news.title: 
      
    # typecaste each val to string 
    val = str(val) 
  
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    comment_words += " ".join(tokens)+" "
split_it = comment_words.split()
Counter = Counter(split_it)
number=20
most_occur = Counter.most_common(number)
most_occur

[('bitcoin', 6996),
 ('price', 5112),
 ('analysis', 3513),
 ('technical', 1946),
 ('ethereum', 1446),
 ('watch', 1200),
 ('blockchain', 1169),
 ('crypto', 885),
 ('cryptocurrency', 698),
 ('new', 595),
 ('btc', 563),
 ('ethusd', 537),
 ('trading', 498),
 ('exchange', 491),
 ('cash', 481),
 ('litecoin', 470),
 ('ripple', 414),
 ('tech', 406),
 ('weekly', 381),
 ('bank', 373)]

In [ ]:
dfm = pd.DataFrame(most_occur, columns = ['Word', 'Count'])
dfm.plot.bar(x='Word',y='Count')

In [63]:
new_news['title']=new_news['title'].apply(lambda x:stemming(x))

In [37]:
new_news

,url,title,text,html,author,source,date
508,https://www.coindesk.com/bitcoin-exchange-btc-...,bitcoin exchange btc china temporarily halts t...,BTC China has become the world's first major b...,<p>BTC China has become the world's first majo...,Emily Spaven,news,2013-01-01 00:00:00+00:00
271,https://www.coindesk.com/what-happened-at-the-...,happened us regulators bitcoin meeting,The Bitcoin Foundation's meeting with represen...,"<p>The Bitcoin Foundation's <a href=""https://w...",Emily Spaven,news,2013-01-01 00:00:00+00:00
593,https://www.coindesk.com/vault-satoshi-expands...,vault satoshi expands canadian bitcoin exchang...,"A new Canadian exchange launched this week, of...","<p>A new Canadian exchange launched this week,...",Danny Bradbury,news,2013-01-02 00:00:00+00:00
175,https://www.coindesk.com/what-will-gambling-do...,gambling bitcoin block chain,What are gambling transactions doing to the bl...,<p>What are gambling transactions doing to the...,Danny Bradbury,news,2013-01-22 00:00:00+00:00
616,https://www.coindesk.com/bitmarkers-move-money...,bitmarkers wants move money bitcoin incasino g...,"At first blush, bitcoin and gambling seem like...","<p>At first blush, bitcoin and gambling seem l...",Daniel Cawrey,news,2013-01-26 00:00:00+00:00
...,...,...,...,...,...,...,...
39158,https://cointelegraph.com/news/fifth-largest-c...,fifth largest crypto exchange bithumb reopen a...,"Bithumb, one of the largest South Korean crypt...","<p dir=""ltr""><a data-amp=""https://cointelegrap...",William Suberg,regulation_news,2018-08-29 00:00:00+00:00
31652,https://www.ccn.com/bitcoin-price-intraday-ana...,bitcoin price intraday analysis btcusd locked ...,Bitcoin price today remained in a bias-conflic...,<p>Bitcoin price today remained in a bias-conf...,Yashu Gola,bitcoin_analysis,2018-08-29 00:00:00+00:00
29949,https://www.newsbtc.com/2018/08/29/ripple-xrp-...,ripple xrp price watch rejected range resistance,Ripple Price Key HighlightsRipple has been on ...,"<span class=""entry-content""><p><strong>Ripple...",Sarah Jenn,analysis,2018-08-29 00:00:00+00:00
28111,https://www.ccn.com/architect-launches-bitcoin...,architect launches bitcoin hostel near rio de ...,"Alessandro Santos, a Brazilian architect, engi...","<p>Alessandro Santos, a Brazilian architect, e...",Francisco Memoria,accepts_bitcoin,2018-10-01 00:00:00+00:00


In [60]:
new_news.to_csv('../archive/new_news_processed.csv',index=False)

In [62]:
new_news


,url,title,text,html,author,source,date
508,https://www.coindesk.com/bitcoin-exchange-btc-...,bitcoin exchange btc china temporarily halts t...,BTC China has become the world's first major b...,<p>BTC China has become the world's first majo...,Emily Spaven,news,2013-01-01 00:00:00+00:00
271,https://www.coindesk.com/what-happened-at-the-...,happened us regulators bitcoin meeting,The Bitcoin Foundation's meeting with represen...,"<p>The Bitcoin Foundation's <a href=""https://w...",Emily Spaven,news,2013-01-01 00:00:00+00:00
593,https://www.coindesk.com/vault-satoshi-expands...,vault satoshi expands canadian bitcoin exchang...,"A new Canadian exchange launched this week, of...","<p>A new Canadian exchange launched this week,...",Danny Bradbury,news,2013-01-02 00:00:00+00:00
175,https://www.coindesk.com/what-will-gambling-do...,gambling bitcoin block chain,What are gambling transactions doing to the bl...,<p>What are gambling transactions doing to the...,Danny Bradbury,news,2013-01-22 00:00:00+00:00
616,https://www.coindesk.com/bitmarkers-move-money...,bitmarkers wants move money bitcoin incasino g...,"At first blush, bitcoin and gambling seem like...","<p>At first blush, bitcoin and gambling seem l...",Daniel Cawrey,news,2013-01-26 00:00:00+00:00
...,...,...,...,...,...,...,...
39158,https://cointelegraph.com/news/fifth-largest-c...,fifth largest crypto exchange bithumb reopen a...,"Bithumb, one of the largest South Korean crypt...","<p dir=""ltr""><a data-amp=""https://cointelegrap...",William Suberg,regulation_news,2018-08-29 00:00:00+00:00
31652,https://www.ccn.com/bitcoin-price-intraday-ana...,bitcoin price intraday analysis btcusd locked ...,Bitcoin price today remained in a bias-conflic...,<p>Bitcoin price today remained in a bias-conf...,Yashu Gola,bitcoin_analysis,2018-08-29 00:00:00+00:00
29949,https://www.newsbtc.com/2018/08/29/ripple-xrp-...,ripple xrp price watch rejected range resistance,Ripple Price Key HighlightsRipple has been on ...,"<span class=""entry-content""><p><strong>Ripple...",Sarah Jenn,analysis,2018-08-29 00:00:00+00:00
28111,https://www.ccn.com/architect-launches-bitcoin...,architect launches bitcoin hostel near rio de ...,"Alessandro Santos, a Brazilian architect, engi...","<p>Alessandro Santos, a Brazilian architect, e...",Francisco Memoria,accepts_bitcoin,2018-10-01 00:00:00+00:00
